In [1]:
import json
import uuid
from pydantic import BaseModel, Field
from typing import List



In [18]:
# ----------------------------------------------------
# Definição dos Modelos Pydantic
# ----------------------------------------------------
class Categoria(BaseModel):
    nome: str
    objetos: List[str]

class KnowledgeObjectFile(BaseModel):
    nome: str
    disciplina: str
    categorias: List[Categoria]
    id: str = None  # Campo extra para armazenar ID

class KnowledgeObjectListFile(BaseModel):
    # Usamos alias para manter a chave original com acento
    areas_de_competencia: List[KnowledgeObjectFile] = Field(alias="áreas_de_competência")


In [16]:
# ----------------------------------------------------
# Função auxiliar para adicionar IDs
# ----------------------------------------------------
def add_ids_to_data(data: KnowledgeObjectListFile) -> KnowledgeObjectListFile:
    # Percorre cada "KnowledgeObjectFile"
    for knowledge_obj in data.areas_de_competencia:
        for categoria in knowledge_obj.categorias:
            novos_objetos = []
            for obj in categoria.objetos:
                novos_objetos.append({"id": str(uuid.uuid4()), "objeto": obj})
            categoria.objetos = novos_objetos
    return data

In [9]:

# ----------------------------------------------------
# Função principal de leitura, adição de id e escrita
# ----------------------------------------------------
def process_json_file(input_file_path: str, output_file_path: str) -> None:
    try:
        # 1. Ler arquivo de entrada
        with open(input_file_path, "r", encoding="utf-8") as f:
            raw_data = json.load(f)

        # 2. Validar/parse com Pydantic
        data_parsed = KnowledgeObjectListFile.model_validate(raw_data)

        # 3. Inserir IDs
        data_with_ids = add_ids_to_data(data_parsed)

        # 4. Gerar dicionário final (mantendo alias de campo)
        data_dict = data_with_ids.model_dump(by_alias=True, exclude_unset=False)

        # 5. Salvar em arquivo de saída
        with open(output_file_path, "w", encoding="utf-8") as f:
            json.dump(data_dict, f, indent=4, ensure_ascii=False)

        print("Arquivo processado com sucesso!")
    except Exception as e:
        print("Erro ao processar o arquivo:", str(e))

In [19]:
process_json_file("data/knowledgeObjects.json", "data/knowledgeObjects_output.json")

Arquivo processado com sucesso!
